IMPORTS:

In [ ]:
# Procesado de datos
# ==============================================================================
import os
import pandas as pd
import numpy as np
from skforecast.datasets import fetch_dataset

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from skforecast.plot import set_dark_theme
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
pio.templates.default = "seaborn"
poff.init_notebook_mode(connected=True)
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

# Keras
# ==============================================================================
os.environ["KERAS_BACKEND"] = "tensorflow" # 'tensorflow', 'jax´ or 'torch'
import keras
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, LSTM

if keras.__version__ > "3.0":
    if keras.backend.backend() == "tensorflow":
        import tensorflow
    elif keras.backend.backend() == "torch":
        import torch
    else:
        print("Backend not recognized. Please use 'tensorflow' or 'torch'.")

# Modelado
# ==============================================================================
import skforecast
from skforecast.ForecasterRnn import ForecasterRnn
from skforecast.ForecasterRnn.utils import create_and_compile_model
from sklearn.preprocessing import MinMaxScaler
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
import sklearn
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

RNN:

In [ ]:
def rnn (data_train,levels,steps,lags): 
    model = create_and_compile_model(
        series=data_train,
        levels=levels, 
        lags=lags,
        steps=steps,
        recurrent_layer="LSTM",
        recurrent_units=4,
        dense_units=16,
        optimizer=Adam(learning_rate=0.01), 
        loss=MeanSquaredError()
    )
    return model

In [ ]:
def forecaster_rnn(data_train,levels,steps,lags,epochs,batch_size,data_val):
    forecaster = ForecasterRnn(
        regressor=rnn(data_train,levels,steps,lags),
        levels=levels,
        transformer_series=MinMaxScaler(),
        fit_kwargs={
            "epochs": epochs,  # Número de épocas para entrenar el modelo.
            "batch_size": batch_size,  # Tamaño del batch para entrenar el modelo.
            "callbacks": [
                EarlyStopping(monitor="val_loss", patience=5)
            ],  # Callback para detener el entrenamiento cuando ya no esté aprendiendo más.
            "series_val": data_val,  # Datos de validación para el entrenamiento del modelo.
        },
    )
    forecaster.fit(data_train)
    return forecaster

In [ ]:
def backtesting_rnn(data,data_train,data_val,levels,steps,lags,epochs,batch_size,end_validation):
    
    forecaster = forecaster_rnn(data_train,levels,steps,lags,epochs,batch_size,data_val)
    metrics, predictions = backtesting_forecaster_multiseries(
        forecaster=forecaster,
        steps=forecaster.max_step,
        series=data,
        levels=forecaster.levels,
        initial_train_size=len(data.loc[:end_validation, :]), # Datos de entrenamiento + validación
        metric="mean_absolute_error",
        verbose=False,
        refit=False,
    )
    for x in levels: 
        rel_mse = 100 * metrics.loc[0, 'mean_absolute_error'] / np.mean(data[x])
        print(f"Media de la serie: {np.mean(data[x]):0.2f}")
        print(f"Error mse relativo: {rel_mse:0.2f} %")
    
    return metrics.loc[0, 'mean_absolute_error']